# Quickstart

In [ ]:
!pip install C:\Users\guisa\Desktop\IMT-LightCurve\dist\imt_lightcurve-1.1-py3-none-any.whl

First, let's import the `LightCurve` class, from imt_lightcurve package, and create a curve object with some data available on [my github](https://github.com/Guilherme-SSB/IC-CoRoT_Kepler/blob/main/03%20-%20Study%20of%20filtering%20processes.ipynb).

In [3]:
from imt_lightcurve.models.lightcurve import LightCurve

Loading BokehJS ...

# Reading and plotting data

In [4]:
import pandas as pd
import numpy as np

# Chosen lightcurve 
LIGHTCURVE = 'RESAMPLED_0101086161_20070516T060226'

# Importing lightcurve data from github
data = pd.read_csv('https://raw.githubusercontent.com/Guilherme-SSB/IC-CoRoT_Kepler/main/resampled_files/' + LIGHTCURVE + '.csv')
time = data.DATE.to_numpy()
flux = data.WHITEFLUX.to_numpy()

# Create the LightCurve object
curve = LightCurve(time=time, flux=flux)

Next, let's plot the curve

In [51]:
print(curve)
curve.plot()

LightCurve Object


and it's Fourier spectrum

In [52]:
curve.view_fourier_spectrum()

## Filtering processes

Everything works great! Nice!

The next step is to filter this lightcurve.

At `LightCurve Object` we have several filtering processes already implemented. The availables ones, with their needed parameters, is:

- Ideal (Cutoff frequency)

- Gaussian (Cutoff frequency)

- Butterworth (Order, Cutoff frequency)

- Bessel (Order, Cutoff frequency, numExpansion=100)

- Median (Number of neighbors)

Let's try some techniques, see the temporal and frequency domain results

In [56]:
butter_o3_f01 = curve.butterworth_lowpass_filter(order=3, cutoff_freq=0.1)
butter_o3_f01.view_filtering_results()

In [57]:
butter_o3_f01.view_fourier_results()

In [58]:
median_nn7 = curve.median_filter(numNei=7)
median_nn7.view_filtering_results()

In [59]:
median_nn7.view_fourier_results()

## Fold curve

In [53]:
folded_curve = curve.fold()
folded_curve.plot()

In purpose of visualization, let's "draw" a median line of this `folded_curve`. 

In [54]:
folded_curve.median_filter(7).view_filtering_results()

## Simulation

Now, let's explore the `Simulate` class. 

First, we are going to import it and create an `SimulationObject`.

In [75]:
from imt_lightcurve.simulation.simulation import Simulate

SimulationObject = Simulate()

The limb darkening constants already defined are:

In [76]:
print(SimulationObject)

Simulate(gamma1=0.44, gamma2=0.23)


### Loading some parameters ...

In [5]:
# Loading folded curve

# Complete path to the folded-observed curve
observed_curve_path = r'C:\Users\guisa\Google Drive\01 - Iniciação Científica\IC-CoRoT_Kepler\Light Curve Simulation\files\curva_luz_eclipse_medio_ID100725706_Butterworth_n2_f02_autocalibrada.txt'
vetor = [x.split('\n')[0] for x in open(observed_curve_path).readlines()]

Nobs = 0
for line in vetor:
    Nobs += 1

observed_curve = np.zeros((Nobs, 3))
time = []
flux = []
flux_error = []

for line in vetor:
    splitted_line = line.split(' ')
    splitted_line_iterator = filter(None, splitted_line)
    splitted_line_filtered = list(splitted_line_iterator)
    time.append(splitted_line_filtered[0])
    flux.append(splitted_line_filtered[1])
    flux_error.append(splitted_line_filtered[2])

observed_curve[:, 0] = time
observed_curve[:, 1] = flux
observed_curve[:, 2] = flux_error

observed_curve_lc = LightCurve(time=observed_curve[:, 0], flux=observed_curve[:, 1], flux_error=observed_curve[:, 2])
observed_curve_lc.plot()

In [4]:
# Loading x values

# Planet coordinate, along the x-axis, as a function of the start's radius
x_values_path = r'C:\Users\guisa\Google Drive\01 - Iniciação Científica\IC-CoRoT_Kepler\Light Curve Simulation\files\Valores_x_simulacao.txt'
x_values = np.loadtxt(x_values_path, dtype='float', delimiter='\n')

### Simulate a lightcurve

In [79]:
# Defining parameters

b_impact = 0.92
p = 0.1483
period = 13.240160
adivR = 22.36

In [80]:
simulated_curve = SimulationObject.simulate_lightcurve(observed_curve=observed_curve_lc, b_impact=b_impact, p=p, period=period, adivR=adivR, x_values=x_values)


Building the light curve...


In [81]:
simulated_curve.view_simulation_results()

Plotting simulation results


In [82]:
simulated_curve.compare_results(see_values=True)

╒═════════════════╤══════════════════╕
│   Original flux │   Simulated flux │
╞═════════════════╪══════════════════╡
│        0.999889 │         1        │
├─────────────────┼──────────────────┤
│        0.999862 │         0.999988 │
├─────────────────┼──────────────────┤
│        0.999841 │         1.00001  │
├─────────────────┼──────────────────┤
│        0.999824 │         1.00001  │
├─────────────────┼──────────────────┤
│        0.999833 │         0.999986 │
├─────────────────┼──────────────────┤
│        0.99984  │         1        │
├─────────────────┼──────────────────┤
│        0.999826 │         1.00002  │
├─────────────────┼──────────────────┤
│        0.999792 │         0.999986 │
├─────────────────┼──────────────────┤
│        0.999769 │         0.999991 │
├─────────────────┼──────────────────┤
│        0.999767 │         1.00003  │
├─────────────────┼──────────────────┤
│        0.999731 │         0.999991 │
├─────────────────┼──────────────────┤
│        0.999639 │      

28.258803989477244

### Simulate values

In [1]:
import numpy as np

In [2]:
# Loading parameters

# Transit impact parameter
b_values_path = r'C:\Users\guisa\Google Drive\01 - Iniciação Científica\IC-CoRoT_Kepler\Light Curve Simulation\files\Valores_b_simulacao_rodada2.txt'
b_values = np.loadtxt(b_values_path, dtype='float', delimiter='\n')

# Radius values of the planet compared to the star
p_values_path = r'C:\Users\guisa\Google Drive\01 - Iniciação Científica\IC-CoRoT_Kepler\Light Curve Simulation\files\Valores_p_simulacao_rodada2.txt'
p_values = np.loadtxt(p_values_path, dtype='float', delimiter='\n')

# Orbital period values to be considered
period_values_path = r'C:\Users\guisa\Google Drive\01 - Iniciação Científica\IC-CoRoT_Kepler\Light Curve Simulation\files\Valores_periodo_simulacao_rodada2.txt'
period_values = np.loadtxt(period_values_path, dtype='float', delimiter='\n')

# Orbital radius values compared to star radius
adivR_values_path = r'C:\Users\guisa\Google Drive\01 - Iniciação Científica\IC-CoRoT_Kepler\Light Curve Simulation\files\Valores_adivR_simulacao_rodada2.txt'
adivR_values = np.loadtxt(adivR_values_path, dtype='float', delimiter='\n')

In [6]:
adivR_values


array([18. , 18.1, 18.2, 18.3, 18.4, 18.5, 18.6, 18.7, 18.8, 18.9, 19. ,
       19.1, 19.2, 19.3, 19.4, 19.5, 19.6, 19.7, 19.8, 19.9, 20. , 20.1,
       20.2, 20.3, 20.4, 20.5, 20.6, 20.7, 20.8, 20.9, 21. , 21.1, 21.2,
       21.3, 21.4, 21.5, 21.6, 21.7, 21.8, 21.9, 22. , 22.1, 22.2, 22.3,
       22.4, 22.5, 22.6, 22.7, 22.8, 22.9, 23. ])

In [85]:
SimulationObject = Simulate()

final_table_sorted_by_chi2 = SimulationObject.simulate_values(observed_curve=observed_curve_lc, b_values=b_values, p_values=p_values, period_values=period_values, adivR_values=adivR_values, x_values=x_values, results_to_csv=False)

Starting simulation...


Simulating: 100%|██████████| 457776/457776 [08:53<00:00, 858.32it/s]



Best parameters are:
-> Best b_impact = 0.92
-> Best p = 0.14
-> Best period = 13.0
-> Best adivR = 18.4
-> Best chi2 = 9.92326701414888



In [88]:
print(final_table_sorted_by_chi2.head())

       b_impact     p  period  adivR      chi2
93640      0.92  0.14   13.00   18.4  9.923267
95368      0.92  0.14   13.33   22.9  9.923267
95369      0.92  0.14   13.33   23.0  9.923267
95370      0.92  0.14   13.34   18.0  9.923267
95371      0.92  0.14   13.34   18.1  9.923267


In [87]:
## Simule a lightcurve with the best fitting values
best_simulated_curve = SimulationObject.simulate_lightcurve(observed_curve=observed_curve_lc, x_values=x_values)
best_simulated_curve.view_simulation_results()
best_simulated_curve.compare_results(see_values=False)


Building the light curve...

Using the best b_impact, computed earlier
Using the best p, computed earlier
Using the best period, computed earlier
Using the best adivR, computed earlier

Plotting simulation results



Chi squared = 9.9233


9.92326701414888